

---



### 1. Instalação e importação das dependências utilizadas no projeto.

In [2]:
!pip install kaggle pandas prefect dbt-sqlite requests opendatasets --upgrade --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.2/85.2 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.1/56.1 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.4/114.4 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.5/58.5 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.4/256.4 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 110.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 72.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 49.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 90.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 67.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.5/31

In [3]:
import pandas as pd
import requests
import json
import sqlite3
import opendatasets as od
import os


print("Ambiente configurado com sucesso!")


Ambiente configurado com sucesso!


### 2. Conexão com a API do Kaggle (requere conta e token de autenticação; passos descritos nos comentários abaixo).






In [4]:


# ------------------------------------------------------------
# 🔐 Gerando o token da API do Kaggle
#
# 1. Acesse: https://www.kaggle.com/account
# 2. Vá em "API"
# 3. Clique em "Create New API Token"
#
# Isso baixa o arquivo kaggle.json com:
# {
#     "username": "seu_usuario",
#     "key": "9b788fbf9b375d262c301e0f8af7c42d"
# }
#
# 4. Copie o conteúdo (username e key), pois o opendatasets
#    vai pedir na primeira execução.
# ------------------------------------------------------------


# 🔗 URL da competição no Kaggle
dataset_url = 'https://www.kaggle.com/competitions/kaggle-survey-2022/data'

# 📥 Download dos dados
# Ao rodar, ele vai pedir:
#   Your Kaggle username:
#   Your Kaggle Key:
# Preencha com as informações do kaggle.json
od.download(dataset_url)


Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: camilesantana
Your Kaggle Key: ··········


100%|██████████| 2.20M/2.20M [00:00<00:00, 118MB/s]


Extracting archive ./kaggle-survey-2022/kaggle-survey-2022.zip to ./kaggle-survey-2022


### 3. Verificação dos arquivos obtidos via Kaggle e exploração dos diretórios disponíveis.

In [5]:
os.listdir('kaggle-survey-2022')

['kaggle_survey_2022_responses.csv', 'Supplementary Data']

### 4. Carregamento da base do Kaggle no Pandas e filtragem dos registros referentes ao gênero feminino.

In [7]:

# Carregar o dataset
df = pd.read_csv(
    'kaggle-survey-2022/kaggle_survey_2022_responses.csv',
    low_memory=False
)

# Filtrar somente mulheres
df_filtrado = df[df["Q3"] == "Woman"]

### 5. Normalização da base: seleção das colunas relevantes e padronização dos nomes.

In [8]:
# ================================
# 1. Criar colunas combinadas
# ================================

# Q12 -> linguagens usadas (1 até 15)
q12_cols = [f"Q12_{i}" for i in range(1, 16)]
df_filtrado["linguagens_que_usa"] = df_filtrado[q12_cols].apply(
    lambda row: ", ".join(
        [str(v) for v in row.values if pd.notnull(v) and v not in ["", 0, False]]
    ),
    axis=1
)

# Q35 -> Ferramentas de ML usadas (1 até 16)
q35_cols = [f"Q35_{i}" for i in range(1, 17)]
df_filtrado["ferramentas_database"] = df_filtrado[q35_cols].apply(
    lambda row: ", ".join(
        [str(v) for v in row.values if pd.notnull(v) and v not in ["", 0, False]]
    ),
    axis=1
)

# Q36 -> Tecnologias em produção (1 até 15)
q36_cols = [f"Q36_{i}" for i in range(1, 16)]
df_filtrado["ferramentas_bi"] = df_filtrado[q36_cols].apply(
    lambda row: ", ".join(
          [str(v) for v in row.values if pd.notnull(v) and v not in ["", 0, False]]
    ),
    axis=1
)

# ================================
# 2. Selecionar colunas finais
# ================================
tabela = df_filtrado[
    [
        "Q2", "Q4", "Q5", "Q8", "Q11", "Q29",
        "linguagens_que_usa",
        "ferramentas_database",
        "ferramentas_bi"
    ]
]

# ================================
# 3. Renomear colunas
# ================================
tabela = tabela.rename(columns={
    "Q2": "faixa_idade",
    "Q4": "pais",
    "Q5": "ind_estudo",
    "Q8": "nivel_estudo",
    "Q11": "tempo_carreira",
    "Q29": "faixa_renda",
})

# Visualizar primeiras linhas
tabela.head(1000)


/tmp/ipython-input-558060802.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtrado["linguagens_que_usa"] = df_filtrado[q12_cols].apply(
/tmp/ipython-input-558060802.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtrado["ferramentas_database"] = df_filtrado[q35_cols].apply(
/tmp/ipython-input-558060802.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

,faixa_idade,pais,ind_estudo,nivel_estudo,tempo_carreira,faixa_renda,linguagens_que_usa,ferramentas_database,ferramentas_bi
6,18-21,India,Yes,Master’s degree,1-3 years,NaN,"Python, SQL",,
22,25-29,China,Yes,Master’s degree,< 1 years,NaN,"Python, MATLAB",,
23,35-39,India,No,Master’s degree,3-5 years,NaN,"Python, Java, MATLAB",,
28,60-69,Argentina,No,Master’s degree,1-3 years,"3,000-3,999",R,,
41,22-24,Canada,No,Master’s degree,5-10 years,"5,000-7,499","Python, SQL",,
...,...,...,...,...,...,...,...,...,...
4781,30-34,India,No,Bachelor’s degree,I have never written code,"10,000-14,999",,,
4782,22-24,India,No,Master’s degree,I have never written code,$0-999,,,
4784,50-54,Ireland,No,Master’s degree,1-3 years,"60,000-69,999","Python, R","MySQL , Oracle Database , Microsoft SQL Server",
4785,22-24,United Arab Emirates,No,Bachelor’s degree,1-3 years,"20,000-24,999",Python,,


### 6. Criação do banco SQLite, definição da tabela participantes e inserção dos dados iniciais.

In [10]:
conn_bootcamp = sqlite3.connect('bootcampBI.db')
cursor_bootcamp = conn_bootcamp.cursor()

cursor_bootcamp.execute("""
    DROP TABLE IF EXISTS participantes;
""")

cursor_bootcamp.execute("""
    CREATE TABLE IF NOT EXISTS participantes (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        nome varchar(150),
        pais varchar(150)
    )
""")

participantes = [
    ("Carla Silva", "Brasil"),
    ("Emily Johnson", "Estados Unidos"),
    ("Yuki Tanaka", "Japão"),
    ("Maria Garcia", "Espanha"),
    ("Wanda Müller", "Alemanha"),
    ("Fatima Al-Fayed", "Egito"),
    ("Camille O'Connor", "Irlanda"),
    ("Chen Wei", "China"),
    ("Amélie Dubois", "França"),
    ("Sofia Rossi", "Itália")
]

cursor_bootcamp.executemany("""
    INSERT INTO participantes (nome, pais)
    VALUES (?, ?)
""", participantes)

conn_bootcamp.commit()
conn_bootcamp.close()

print("Dados inseridos com sucesso!")

Dados inseridos com sucesso!


### 7. Criação do arquivo JSON com a categorização das habilidades técnicas.

In [12]:
# %%writefile é uma *cell magic* do Jupyter/IPython.
# Tudo que estiver escrito nessa célula será SALVO em um arquivo.
# Não é Python “puro”, funciona apenas dentro de notebooks (Jupyter / Colab).
#
# Neste caso, estamos criando (ou sobrescrevendo) um arquivo chamado:
# "habilidades_categorias.json"
#
# Arquivos .json são usados para armazenar dados estruturados
# em formato de chave:valor, muito comum em APIs e projetos de dados.

%%writefile habilidades_categorias.json
{
  "Ferramentas de Análise de Dados": ["Python", "R", "SQL" ],
  "Ferramentas de BI": ["Tableau", "Power BI", "Looker" ],
  "Ferramentas de Nuvem": ["AWS", "Google Cloud", "Microsoft Azure"]
}

Overwriting habilidades_categorias.json


### 8. Inicialização da conexão com o Data Warehouse em SQLite.

In [13]:
conn_datawarehouse = sqlite3.connect('datawarehouse.db')
conn_datawarehouse.close()

print("Data Warehouse criado com sucesso!")

Data Warehouse criado com sucesso!
